In [ ]:
import json
import pandas
from datetime import datetime, timedelta
import re
from pathlib import Path

In [ ]:
file = 'rss.txt'
out_file = Path(file).with_suffix('.json')
data = pandas.read_xml(file, xpath=".//item")
data.drop(data[data["title"] == "Weekly Meeting"].index, inplace=True)

In [ ]:
expression = re.compile('.*<p><p>(?P<description>.*)<\/p> <\/p> <p>(?P<location>.*)<\/p> <p>(?P<date>.*)<\/p> <p>(?P<attendees>.*)<\/p> <p>Price: (?P<price>[0-9]+).*<\/p> <p>(?P<url>.*)<\/p>.*')

In [ ]:
output = pandas.DataFrame()

for index, row in data.iterrows():
    description_groups = expression.match(row["description"]).groupdict()
    date_time = datetime.strptime(description_groups['date'], '%A, %B %d at %I:%M %p').replace(year=datetime.today().year)
    temp = {"title": row['title'],
			"subtitle": "",
			"description": description_groups['description'].replace('</p>','').replace('<p>',''),
			"date": date_time.strftime('%Y-%m-%d'),
			"start_time": date_time.strftime('%-I:%M%p').lower(),
			"end_time": (date_time + timedelta(hours=3)).strftime('%-I:%M%p').lower(),
			"offsite_location": 'null',
			"offered_online": 'false',
			"offered_in_person": 'true',
			"member_price": str(int(description_groups['price'])-5),
			"non_member_price": description_groups['price'],
			"image": "",
			"meetup_link": row['guid']}
    output = output.append(temp, ignore_index=True)

In [ ]:
with open(out_file,'w') as f:
    f.write(json.dumps(json.loads(output.to_json(orient="records"))))